# TOOLS

This notebook contains different sections that help to:
- Generate different video variations 
- Compute metrics using external software like ffmpeg or libav

It makes use of different scripts located in the /scripts folder that will help to both generate different renditions from original (1080p) input videos as well as distortions of them (attacks).

It also provides means to execute metric extractions from those videos (MS-SSIM, VMAF, SSIM and PSNR) by means of bash shell. These metrics are also utilized in other notebooks.

# Input path configuration

The cell below must be executed. It prepares the notebook to point to the local repo structure where the original 1080p renditions are.

The expected input structure is:

```
data
├── 1080p
│   └── 01.mp4
├── 720p
│   └── 01.mp4
├── 480p
│   └── 01.mp4
├── 360p
│   └── 01.mp4
└── 240p
|    └── 01.mp4
└── 144p
    └── 01.mp4    
```

In [ ]:
import os.path
from pathlib import Path

current_path = Path(os.path.abspath(''))
input_path = (current_path / "../data/1080p").resolve()

# 1.-Rendition creation
## 1.0.- Scaling and bitrate reduction

In first place, a dataset with original videos is required. We propose the collection provided in the YT8M notebook available [here](https://github.com/epiclabs-io/YT8M/blob/master/yt8m-crawler.ipynb). This would provide a nice collection of renditions at different resolutions and with adequate bitrates. Nevertheless, feel free to supply your own set of videos and renditions at whatever bitrate you consider.

## 1.1.- Watermarks

There is a python script in order to insert watermarks in the videos. This script receives 4 parameters:
- The input path (-i or --input) which is the folder containing 1080p.
- The output path (-o or --output) which is the folder where the videos with watermark are going to be sotred.
- The metadata (-m or --metadata) which is the file containing data about the videos, the most important is the needed bitrate to enconde the video.
- The watermark file (-w --watermark) which is the file containing the image to be applied to the video.

The output looks like

```
├── 1080p_watermark
│   ├── 01.mp4
├── 720p_watermark
│   ├── 01.mp4
├── 480p_watermark
│   ├── 01.mp4
├── 360p_watermark
│   ├── 01.mp4
├── 240p_watermark
│   ├── 01.mp4
├── 144p_watermark
│   ├── 01.mp4
```

In [ ]:
output_path = (current_path / "../data").resolve()
metadata_file = (current_path / "/scripts/yt8m_data.csv").resolve()
watermark_file = (current_path / "/scripts/watermark/livepeer.png").resolve()
%run -i '/scripts/watermark.py' -i $input_path -o $output_path -m $metadata_file -w $watermark_file

## 1.2.- Flips / rotations

There is a python script in order to flip / rotate the videos. This script receives 4 parameters:
- The input path (-i or --input) which is the folder containing 1080p.
- The output path (-o or --output) which is the folder where the videos with watermark are going to be sotred.
- The metadata (-m or --metadata) which is the file containing data about the videos, the most important is the needed bitrate to enconde the video.
- The desired flip or rotation:
    -  -vf or --vflip for the vertical flip
    -  -hf or --hflip for the horizontal flip
    -  -cf or for the 90 degrees clockwise rotation
    -  -ccf for the 90 degrees counterclockwise rotation

There are implemented the following ways to flip / rotate a video:

## 1.3.- Vertical flip

The output looks like

```
├── 1080p_flip_vertical
│   ├── 01.mp4
├── 720p_flip_vertical
│   ├── 01.mp4
├── 480p_flip_vertical
│   ├── 01.mp4
├── 360p_flip_vertical
│   ├── 01.mp4
├── 240p_flip_vertical
│   ├── 01.mp4
├── 144p_flip_vertical
│   ├── 01.mp4
```

In [ ]:
output_path = (current_path / "../data").resolve()
metadata_file = (current_path / "/scripts/yt8m_data.csv").resolve()
%run -i '/scripts/flip.py' -i $input_path -o $output_path -m $metadata_file -vf

## 1.4.- Horizontal flip

The output looks like

```
├── 1080p_flip_horizontal
│   ├── 01.mp4
├── 720p_flip_horizontal
│   ├── 01.mp4
├── 480p_flip_horizontal
│   ├── 01.mp4
├── 360p_flip_horizontal
│   ├── 01.mp4
├── 240p_flip_horizontal
│   ├── 01.mp4
├── 144p_flip_horizontal
│   ├── 01.mp4
```

In [ ]:
output_path = (current_path / "../data").resolve()
metadata_file = (current_path / "/scripts/yt8m_data.csv").resolve()
%run -i '/scripts/flip.py' -i $input_path -o $output_path -m $metadata_file -hf

## 1.5.- Rotate 90 degrees clockwise

The output looks like

```
├── 1080p_rotate_90_clockwise
│   ├── 01.mp4
├── 720p_rotate_90_clockwise
│   ├── 01.mp4
├── 480p_rotate_90_clockwise
│   ├── 01.mp4
├── 360p_rotate_90_clockwise
│   ├── 01.mp4
├── 240p_rotate_90_clockwise
│   ├── 01.mp4
├── 144p_rotate_90_clockwise
│   ├── 01.mp4
```

In [ ]:
output_path = (current_path / "../data").resolve()
metadata_file = (current_path / "/scripts/yt8m_data.csv").resolve()
%run -i '/scripts/flip.py' -i $input_path -o $output_path -m $metadata_file -cf

## 1.6.- Rotate 90 degrees counterclockwise

The output looks like

```
├── 1080p_rotate_90_counterclockwise
│   ├── 01.mp4
├── 720p_rotate_90_counterclockwise
│   ├── 01.mp4
├── 480p_rotate_90_counterclockwise
│   ├── 01.mp4
├── 360p_rotate_90_counterclockwise
│   ├── 01.mp4
├── 240p_rotate_90_counterclockwise
│   ├── 01.mp4
├── 144p_rotate_90_counterclockwise
│   ├── 01.mp4
```

In [ ]:
output_path = (current_path / "../data").resolve()
metadata_file = (current_path / "/scripts/yt8m_data.csv").resolve()
%run -i '/scripts/flip.py' -i $input_path -o $output_path -m $metadata_file -ccf

# 2 Metric extraction

In order to extract metrics from the videos above and being able to compare them, a set of scripts are made available below.

## 2.1 Compute ms-ssim

There is a bash script `evaluate-ms-ssim.sh` in order to calculate the multiscale ssim. This script receives two parameters which are the path where the videos are and the path where the output should be placed. Note that the `../output` folder is the folder where other notebooks are expecting the output.


The output structuture is inside the output folder

```
mssim
├── 240
│   ├── 01
│   │   └── 01_240.log
├── 360
│   ├── 01
│   │   └── 01_360.log
├── 480
│   ├── 01
│   │   └── 01_480.log
└── 720
    ├── 01
        └── 01_720.log 
```

Where the folder indicates the rendition we are using to compare against the original (1080p). 
A subfolder of this folder contains the name of the asset and finally the file containing the log.

The log is a csv file, with the following structure:

```
ms-ssim, psnr-y, psnr-u, psnr-v
0.986889, 32.866684, 43.274622, 42.429359
0.985558, 32.394349, 43.344157, 42.658971
0.985460, 32.521368, 43.338460, 42.580399
0.985896, 32.670122, 43.325404, 42.529248
```

In [ ]:
shell_path = Path(os.path.abspath(''))
shell_input_path = (current_path / "../data").resolve()
shell_output_path = (current_path / "../output").resolve()

!bash '/scripts/shell/evaluate-ms-ssim.sh' $shell_input_path $shell_output_path

## 2.1.- Compute VMAF

There is a bash script `evaluate-vmaf.sh` in order to calculate the vmaf score. This script receives two parameters which are the path where the videos are and the path where the output should be placed. Note that the `../output` folder is the folder where other notebooks are expecting the output.

The script will produce the following folder structure:

```
output/vmaf
├── 240
│   ├── 01
│   │   ├── 01_240.log
│   │   └── 01_240.log.out
├── 360
│   ├── 01
│   │   ├── 01_360.log
│   │   └── 01_360.log.out
├── 480
│   ├── 01
│   │   ├── 01_480.log
│   │   └── 01_480.log.out
└── 720
    ├── 01
        ├── 01_720.log
        └── 01_720.log.out
```

Where the folder indicates the rendition we are using to compare against the original (1080p). 
A subfolder of this folder contains the name of the asset and finally two files: One containing the result 
(videoname_rendition_resolution.log) and other containing the output from the ffmpeg (videoname_rendition_resolution.log.out).

The log file contains the following information:

```
Start calculating VMAF score...
Exec FPS: 158.922597
VMAF score = 90.566873
```

The interesting line is the third one, containing the vmaf score.

The .out file is not worth analyzing as it is the standard ffmpeg output

In [ ]:
shell_path = Path(os.path.abspath(''))
shell_input_path = (current_path / "../data").resolve()
shell_output_path = (current_path / "../output").resolve()

!bash '/scripts/shell/evaluate-vmaf.sh' $shell_input_path $shell_output_path

## 2.3.- Compute ssim and psnr

There is a bash script `evaluate-psnr-ssim.sh` in order to calculate the ssim and psnr metrics. This script receives two parameters which are the path where the videos are and the path where the output should be placed. Note that the `../output` folder is the folder where other notebooks are expecting the output.

The script will produce the following folder structure:

The output structuture is inside the output folder for the psnr

```
psnr
├── 240
│   ├── 01
│   │   └── 01_240.log
├── 360
│   ├── 01
│   │   └── 01_360.log
├── 480
│   ├── 01
│   │   └── 01_480.log
└── 720
    ├── 01
        └── 01_720.log 
```

Where the folder indicates the rendition we are using to compare against the original (1080p). 
A subfolder of this folder contains the name of the asset and finally the file containing the log.

The log has the following structure:

n:1 mse_avg:0.60 mse_y:0.73 mse_u:0.34 mse_v:0.32 psnr_avg:50.37 psnr_y:49.50 psnr_u:52.76 psnr_v:53.01 
n:2 mse_avg:0.83 mse_y:1.01 mse_u:0.48 mse_v:0.45 psnr_avg:48.95 psnr_y:48.09 psnr_u:51.29 psnr_v:51.62 
n:3 mse_avg:0.77 mse_y:0.94 mse_u:0.45 mse_v:0.44 psnr_avg:49.25 psnr_y:48.42 psnr_u:51.59 psnr_v:51.71 
n:4 mse_avg:0.76 mse_y:0.92 mse_u:0.45 mse_v:0.43 psnr_avg:49.32 psnr_y:48.50 psnr_u:51.55 psnr_v:51.76 
n:5 mse_avg:0.65 mse_y:0.79 mse_u:0.39 mse_v:0.36 psnr_avg:50.01 psnr_y:49.18 psnr_u:52.22 psnr_v:52.58 


The output structuture is inside the output folder for the ssim

```
ssim
├── 240
│   ├── 01
│   │   └── 01_240.log
├── 360
│   ├── 01
│   │   └── 01_360.log
├── 480
│   ├── 01
│   │   └── 01_480.log
└── 720
    ├── 01
        └── 01_720.log 
```

Where the folder indicates the rendition we are using to compare against the original (1080p). 
A subfolder of this folder contains the name of the asset and finally the file containing the log.



In [ ]:
shell_path = Path(os.path.abspath(''))
shell_input_path = (current_path / "../data").resolve()
shell_output_path = (current_path / "../output").resolve()

!bash '/scripts/shell/evaluate-psnr-ssim.sh' $shell_input_path $shell_output_path